URLs of data

In [2]:
arrondissements_url = 'https://raw.githubusercontent.com/mariobecerra/mda_project/main/data/arrondissements_coords.csv'
mortality_url = 'https://raw.githubusercontent.com/mariobecerra/mda_project/main/data/mortality_data_2000-2019.csv'
temperature_url = 'https://raw.githubusercontent.com/mariobecerra/mda_project/main/data/temp_2000_2019.csv'



In [8]:
import os
os.getcwd()
os.chdir("C:\\Users\\baris\\Desktop\\Modern Data Analytics\\mda_project")

Read data

In [3]:
import pandas as pd
import numpy as np
import datetime

In [11]:
temperature_1980_2020 = pd.read_csv('./data/T_Belgium_1980-2020.csv', sep=";") #The file is only being read when I upload the file to the notebook session, it is not being read correctly if read from project folder:(
temperature_1980_2020.head(5) 

,GRID_NO,LATITUDE,LONGITUDE,ALTITUDE,DAY,TEMPERATURE_MAX,TEMPERATURE_MIN,TEMPERATURE_AVG
0,101094,50.69632,4.56668,130,19800101,3.5,-2.5,0.5
1,101094,50.69632,4.56668,130,19800102,1.4,-2.8,-0.7
2,101094,50.69632,4.56668,130,19800103,1.4,-1.2,0.1
3,101094,50.69632,4.56668,130,19800104,1.5,-1.1,0.2
4,101094,50.69632,4.56668,130,19800105,4.4,0.9,2.7


In [12]:
arrondissements_data = pd.read_csv(arrondissements_url)

arrondissements_data.head(5)

,NIS_Code,Nom_arrondissement,lat,lon
0,11000,Antwerpen,51.280681,4.505809
1,63000,Verviers,50.464213,6.022070
2,13000,Turnhout,51.250094,4.950422
3,62000,Liège,50.613433,5.607778
4,64000,Waremme,50.662789,5.206501


In [13]:
mortality_data = pd.read_csv(mortality_url)
mortality_data

,YEAR,YEAR_WEEK,COD,ARRON,N_MASK
0,2000,2000-001,external,11000,NaN
1,2000,2000-001,external,12000,NaN
2,2000,2000-001,external,13000,NaN
3,2000,2000-001,external,21000,NaN
4,2000,2000-001,external,23000,NaN
5,2000,2000-001,external,24000,5.0
6,2000,2000-001,external,31000,NaN
7,2000,2000-001,external,34000,NaN
8,2000,2000-001,external,35000,NaN
9,2000,2000-001,external,44000,NaN


In [14]:
temperature_data = pd.read_csv(temperature_url, sep=";")
temperature_data

,GRID_NO,LATITUDE,LONGITUDE,DAY,TEMPERATURE_MAX,TEMPERATURE_MIN,TEMPERATURE_AVG
0,96095,49.59159,5.03811,20000101,6.2,2.9,4.5
1,96095,49.59159,5.03811,20000102,6.0,4.2,5.1
2,96095,49.59159,5.03811,20000103,5.4,3.1,4.3
3,96095,49.59159,5.03811,20000104,8.1,4.4,6.3
4,96095,49.59159,5.03811,20000105,8.1,1.8,4.9
5,96095,49.59159,5.03811,20000106,8.6,1.8,5.2
6,96095,49.59159,5.03811,20000107,7.5,5.0,6.3
7,96095,49.59159,5.03811,20000108,5.7,2.5,4.1
8,96095,49.59159,5.03811,20000109,5.5,4.1,4.8
9,96095,49.59159,5.03811,20000110,4.0,0.0,2.0


Find the closest temperature grid to each arrondissement

In [16]:
grids_temp = temperature_1980_2020.drop_duplicates(subset = ['GRID_NO', 'LATITUDE', 'LONGITUDE'])
grids_temp

,GRID_NO,LATITUDE,LONGITUDE,ALTITUDE,DAY,TEMPERATURE_MAX,TEMPERATURE_MIN,TEMPERATURE_AVG
0,101094,50.69632,4.56668,130,19800101,3.5,-2.5,0.5
14976,101095,50.71245,4.91980,111,19800101,2.7,-1.8,0.4
29952,101096,50.72749,5.27314,128,19800101,2.0,-1.3,0.3
44928,102094,50.92034,4.54048,42,19800101,4.1,-1.9,1.1
59904,102095,50.93656,4.89528,60,19800101,2.7,-2.2,0.2
74880,102096,50.95169,5.25031,29,19800101,2.2,-1.6,0.3


In [17]:
# Not the most efficient, but it's a small dataset so it's okay
distances = pd.DataFrame()

for i in range(arrondissements_data.shape[0]):
  diff_lat = grids_temp['LATITUDE'] - arrondissements_data.loc[i].at['lat']
  diff_lon = grids_temp['LONGITUDE'] - arrondissements_data.loc[i].at['lon']
  distances_i = np.power((diff_lat), 2) + np.power((diff_lon), 2)

  temp = pd.DataFrame(
        {
            'NIS_Code': arrondissements_data.loc[i].at['NIS_Code'],
            'GRID_NO': grids_temp.GRID_NO,
            'Dist': distances_i
        }
    )
  distances = pd.concat([distances, temp])



In [20]:
distances

,NIS_Code,GRID_NO,Dist
0,11000,101094,0.345183
14976,11000,101095,0.494275
29952,11000,101096,0.894818
44928,11000,102094,0.131048
59904,11000,102095,0.270107
74880,11000,102096,0.662517
0,63000,101094,2.172035
14976,63000,101095,1.276622
29952,63000,101096,0.630212
44928,63000,102094,2.403162


Find the temperature grid closest to each arrondissement

In [21]:
arron_grid_mapping = distances.groupby('NIS_Code').apply(lambda x: x[x['Dist'] == x['Dist'].min()])[['NIS_Code', 'GRID_NO']].reset_index(drop = True)

arron_grid_mapping.head(8)

,NIS_Code,GRID_NO
0,11000,102094
1,12000,102094
2,13000,102095
3,21000,102094
4,23000,102094
5,24000,102095
6,25000,101094
7,31000,102094


Join arrondissement and temperature datasets to get temperature per day in each arrondissement.

In [22]:
join_1 = pd.merge(arron_grid_mapping, temperature_1980_2020, how = "inner")
join_1


,NIS_Code,GRID_NO,LATITUDE,LONGITUDE,ALTITUDE,DAY,TEMPERATURE_MAX,TEMPERATURE_MIN,TEMPERATURE_AVG
0,11000,102094,50.92034,4.54048,42,19800101,4.1,-1.9,1.1
1,11000,102094,50.92034,4.54048,42,19800102,2.0,-2.2,-0.1
2,11000,102094,50.92034,4.54048,42,19800103,2.0,-0.6,0.7
3,11000,102094,50.92034,4.54048,42,19800104,2.1,-0.5,0.8
4,11000,102094,50.92034,4.54048,42,19800105,5.0,1.5,3.2
5,11000,102094,50.92034,4.54048,42,19800106,5.0,3.4,4.2
6,11000,102094,50.92034,4.54048,42,19800107,6.5,3.9,5.2
7,11000,102094,50.92034,4.54048,42,19800108,5.8,2.4,4.1
8,11000,102094,50.92034,4.54048,42,19800109,4.1,1.9,3.0
9,11000,102094,50.92034,4.54048,42,19800110,3.4,-2.1,0.6


In [23]:
arron_temp_data = pd.merge(join_1, arrondissements_data, how = 'left')[['DAY', 'Nom_arrondissement', 'NIS_Code', 'GRID_NO', 'TEMPERATURE_MAX', 'TEMPERATURE_MIN', 'TEMPERATURE_AVG']].sort_values(by=['Nom_arrondissement', 'NIS_Code', 'DAY']).assign(YEAR = lambda x: np.floor(x.DAY/10000))
arron_temp_data

,DAY,Nom_arrondissement,NIS_Code,GRID_NO,TEMPERATURE_MAX,TEMPERATURE_MIN,TEMPERATURE_AVG,YEAR
179712,19800101,Aalst,41000,102094,4.1,-1.9,1.1,1980.0
179713,19800102,Aalst,41000,102094,2.0,-2.2,-0.1,1980.0
179714,19800103,Aalst,41000,102094,2.0,-0.6,0.7,1980.0
179715,19800104,Aalst,41000,102094,2.1,-0.5,0.8,1980.0
179716,19800105,Aalst,41000,102094,5.0,1.5,3.2,1980.0
179717,19800106,Aalst,41000,102094,5.0,3.4,4.2,1980.0
179718,19800107,Aalst,41000,102094,6.5,3.9,5.2,1980.0
179719,19800108,Aalst,41000,102094,5.8,2.4,4.1,1980.0
179720,19800109,Aalst,41000,102094,4.1,1.9,3.0,1980.0
179721,19800110,Aalst,41000,102094,3.4,-2.1,0.6,1980.0


Get percentiles to define heatwave

In [50]:
arron_temp_data[(arron_temp_data.NIS_Code==13000) & (arron_temp_data.YEAR==1980)].TEMPERATURE_MAX.quantile(0.95)

24.824999999999996

In [24]:
def q95(x):
    return x.quantile(0.95)

temp_percetiles_year_arron = arron_temp_data[['YEAR', 'NIS_Code', 'Nom_arrondissement', 'TEMPERATURE_MAX']].groupby(['YEAR', 'NIS_Code', 'Nom_arrondissement']).agg(q95).stack(level=0).reset_index().rename(columns={0:"p95"})
temp_percetiles_year_arron




,YEAR,NIS_Code,Nom_arrondissement,level_3,p95
0,1980.0,11000,Antwerpen,TEMPERATURE_MAX,26.125
1,1980.0,12000,Mechelen,TEMPERATURE_MAX,26.125
2,1980.0,13000,Turnhout,TEMPERATURE_MAX,24.825
3,1980.0,21000,Bruxelles-Capitale,TEMPERATURE_MAX,26.125
4,1980.0,23000,Hal-Vilvorde,TEMPERATURE_MAX,26.125
5,1980.0,24000,Leuven,TEMPERATURE_MAX,24.825
6,1980.0,25000,Nivelles,TEMPERATURE_MAX,25.525
7,1980.0,31000,Brugge,TEMPERATURE_MAX,26.125
8,1980.0,32000,Diksmuide,TEMPERATURE_MAX,26.125
9,1980.0,33000,Ypres,TEMPERATURE_MAX,26.125


Get temperatures of five consecutive days

In [25]:

temp_lead_1 = arron_temp_data.groupby(['Nom_arrondissement', 'NIS_Code'])['TEMPERATURE_MAX'].shift(-1)

temp_lead_2 = arron_temp_data.groupby(['Nom_arrondissement', 'NIS_Code'])['TEMPERATURE_MAX'].shift(-2)

temp_lead_3 = arron_temp_data.groupby(['Nom_arrondissement', 'NIS_Code'])['TEMPERATURE_MAX'].shift(-3)

temp_lead_4 = arron_temp_data.groupby(['Nom_arrondissement', 'NIS_Code'])['TEMPERATURE_MAX'].shift(-4)

    

In [26]:
arron_temp_data['temp_lead_1'] = temp_lead_1
arron_temp_data['temp_lead_2'] = temp_lead_2
arron_temp_data['temp_lead_3'] = temp_lead_3
arron_temp_data['temp_lead_4'] = temp_lead_4
arron_temp_data

,DAY,Nom_arrondissement,NIS_Code,GRID_NO,TEMPERATURE_MAX,TEMPERATURE_MIN,TEMPERATURE_AVG,YEAR,temp_lead_1,temp_lead_2,temp_lead_3,temp_lead_4
179712,19800101,Aalst,41000,102094,4.1,-1.9,1.1,1980.0,2.0,2.0,2.1,5.0
179713,19800102,Aalst,41000,102094,2.0,-2.2,-0.1,1980.0,2.0,2.1,5.0,5.0
179714,19800103,Aalst,41000,102094,2.0,-0.6,0.7,1980.0,2.1,5.0,5.0,6.5
179715,19800104,Aalst,41000,102094,2.1,-0.5,0.8,1980.0,5.0,5.0,6.5,5.8
179716,19800105,Aalst,41000,102094,5.0,1.5,3.2,1980.0,5.0,6.5,5.8,4.1
179717,19800106,Aalst,41000,102094,5.0,3.4,4.2,1980.0,6.5,5.8,4.1,3.4
179718,19800107,Aalst,41000,102094,6.5,3.9,5.2,1980.0,5.8,4.1,3.4,1.2
179719,19800108,Aalst,41000,102094,5.8,2.4,4.1,1980.0,4.1,3.4,1.2,-0.5
179720,19800109,Aalst,41000,102094,4.1,1.9,3.0,1980.0,3.4,1.2,-0.5,-1.5
179721,19800110,Aalst,41000,102094,3.4,-2.1,0.6,1980.0,1.2,-0.5,-1.5,-1.4


Create boolean variable if there are 5 consecutive days in which temperature was higher than the 95-th percentile of temperature each year and each arrondissement

In [27]:
join_percentiles = pd.merge(arron_temp_data, temp_percetiles_year_arron[['YEAR', 'NIS_Code', 'Nom_arrondissement', 'p95']], how = "left")
join_percentiles

,DAY,Nom_arrondissement,NIS_Code,GRID_NO,TEMPERATURE_MAX,TEMPERATURE_MIN,TEMPERATURE_AVG,YEAR,temp_lead_1,temp_lead_2,temp_lead_3,temp_lead_4,p95
0,19800101,Aalst,41000,102094,4.1,-1.9,1.1,1980.0,2.0,2.0,2.1,5.0,26.125
1,19800102,Aalst,41000,102094,2.0,-2.2,-0.1,1980.0,2.0,2.1,5.0,5.0,26.125
2,19800103,Aalst,41000,102094,2.0,-0.6,0.7,1980.0,2.1,5.0,5.0,6.5,26.125
3,19800104,Aalst,41000,102094,2.1,-0.5,0.8,1980.0,5.0,5.0,6.5,5.8,26.125
4,19800105,Aalst,41000,102094,5.0,1.5,3.2,1980.0,5.0,6.5,5.8,4.1,26.125
5,19800106,Aalst,41000,102094,5.0,3.4,4.2,1980.0,6.5,5.8,4.1,3.4,26.125
6,19800107,Aalst,41000,102094,6.5,3.9,5.2,1980.0,5.8,4.1,3.4,1.2,26.125
7,19800108,Aalst,41000,102094,5.8,2.4,4.1,1980.0,4.1,3.4,1.2,-0.5,26.125
8,19800109,Aalst,41000,102094,4.1,1.9,3.0,1980.0,3.4,1.2,-0.5,-1.5,26.125
9,19800110,Aalst,41000,102094,3.4,-2.1,0.6,1980.0,1.2,-0.5,-1.5,-1.4,26.125


In [28]:
nan_values = join_percentiles[join_percentiles.isna().any(axis=1)]

print (nan_values)

             DAY   Nom_arrondissement  NIS_Code  GRID_NO  TEMPERATURE_MAX  \
14972   20201228                Aalst     41000   102094              5.1   
14973   20201229                Aalst     41000   102094              4.6   
14974   20201230                Aalst     41000   102094              5.4   
14975   20201231                Aalst     41000   102094              3.3   
29948   20201228            Antwerpen     11000   102094              5.1   
29949   20201229            Antwerpen     11000   102094              4.6   
29950   20201230            Antwerpen     11000   102094              5.4   
29951   20201231            Antwerpen     11000   102094              3.3   
44924   20201228                Arlon     81000   101096              4.6   
44925   20201229                Arlon     81000   101096              4.3   
44926   20201230                Arlon     81000   101096              4.5   
44927   20201231                Arlon     81000   101096              3.0   

In [29]:
bool0 = (join_percentiles['TEMPERATURE_MAX'].to_numpy() >= join_percentiles['p95'].to_numpy()).astype(int)
bool1 = (join_percentiles['temp_lead_1'].to_numpy() >= join_percentiles['p95'].to_numpy()).astype(int)
bool2 = (join_percentiles['temp_lead_2'].to_numpy() >= join_percentiles['p95'].to_numpy()).astype(int)
bool3 = (join_percentiles['temp_lead_3'].to_numpy() >= join_percentiles['p95'].to_numpy()).astype(int)
bool4 = (join_percentiles['temp_lead_4'].to_numpy() >= join_percentiles['p95'].to_numpy()).astype(int)
heatwave_boolean = bool0 * bool1 * bool2 * bool3 * bool4
heatwave_boolean

C:\Users\baris\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in greater_equal
  
C:\Users\baris\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater_equal
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\baris\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in greater_equal
  after removing the cwd from sys.path.
C:\Users\baris\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in greater_equal
  """


array([0, 0, 0, ..., 0, 0, 0])

In [30]:
# Not the most efficient, but it works
iso_weeks = ['1990-001']*join_percentiles.shape[0] # Prefill list with correct size
for i in range(len(iso_weeks)):
  day_i = join_percentiles.loc[i].at['DAY']
  iso_week_i = datetime.datetime.strptime(str(day_i), '%Y%m%d').isocalendar()
  iso_weeks[i] = str(iso_week_i[0]) + '-' + str(iso_week_i[1]).zfill(3)



In [31]:
join_percentiles.columns

Index(['DAY', 'Nom_arrondissement', 'NIS_Code', 'GRID_NO', 'TEMPERATURE_MAX',
       'TEMPERATURE_MIN', 'TEMPERATURE_AVG', 'YEAR', 'temp_lead_1',
       'temp_lead_2', 'temp_lead_3', 'temp_lead_4', 'p95'],
      dtype='object')

In [33]:
heat_wave_day_def = join_percentiles[['NIS_Code', 'Nom_arrondissement','YEAR', 'TEMPERATURE_MAX', 'TEMPERATURE_AVG', 'p95']].copy()
heat_wave_day_def['heatwave_boolean'] = heatwave_boolean
heat_wave_day_def['YEAR'] = heat_wave_day_def['YEAR'].astype(int)
heat_wave_day_def

,NIS_Code,Nom_arrondissement,YEAR,TEMPERATURE_MAX,TEMPERATURE_AVG,p95,heatwave_boolean
0,41000,Aalst,1980,4.1,1.1,26.125,0
1,41000,Aalst,1980,2.0,-0.1,26.125,0
2,41000,Aalst,1980,2.0,0.7,26.125,0
3,41000,Aalst,1980,2.1,0.8,26.125,0
4,41000,Aalst,1980,5.0,3.2,26.125,0
5,41000,Aalst,1980,5.0,4.2,26.125,0
6,41000,Aalst,1980,6.5,5.2,26.125,0
7,41000,Aalst,1980,5.8,4.1,26.125,0
8,41000,Aalst,1980,4.1,3.0,26.125,0
9,41000,Aalst,1980,3.4,0.6,26.125,0


In [34]:
heat_wave_day_def['tempdiff_max_p95'] = heat_wave_day_def['TEMPERATURE_MAX'] - heat_wave_day_def['p95'] 
heat_wave_day_def[heat_wave_day_def['YEAR']==1980]

,NIS_Code,Nom_arrondissement,YEAR,TEMPERATURE_MAX,TEMPERATURE_AVG,p95,heatwave_boolean,tempdiff_max_p95
0,41000,Aalst,1980,4.1,1.1,26.125,0,-22.025
1,41000,Aalst,1980,2.0,-0.1,26.125,0,-24.125
2,41000,Aalst,1980,2.0,0.7,26.125,0,-24.125
3,41000,Aalst,1980,2.1,0.8,26.125,0,-24.025
4,41000,Aalst,1980,5.0,3.2,26.125,0,-21.125
5,41000,Aalst,1980,5.0,4.2,26.125,0,-21.125
6,41000,Aalst,1980,6.5,5.2,26.125,0,-19.625
7,41000,Aalst,1980,5.8,4.1,26.125,0,-20.325
8,41000,Aalst,1980,4.1,3.0,26.125,0,-22.025
9,41000,Aalst,1980,3.4,0.6,26.125,0,-22.725


Calculate the heat waves frequency (the number of episodes per year) and the intensity (average T above the local threshold during heat waves) per arrondissement

In [35]:
arrons_iter = heat_wave_day_def.NIS_Code.unique().tolist()
start_yr = int(heat_wave_day_def.YEAR.min())
stop_yr = int(heat_wave_day_def.YEAR.max())
years_iter = [i for i in range(start_yr, stop_yr+1)]
hw_count_df_cols = ['arron'] + years_iter
hw_count_df = pd.DataFrame(columns=hw_count_df_cols)
hw_int_df = pd.DataFrame(columns=hw_count_df_cols)
hw_count_df['arron'] = arrons_iter
hw_int_df['arron'] = arrons_iter

heat_wave_intensity = heat_wave_day_def[heat_wave_day_def['heatwave_boolean'] == 1].copy()
summarized_temp_for_hw_intensity = heat_wave_intensity.groupby(['NIS_Code', 'YEAR'])[['tempdiff_max_p95']].mean().reset_index()

summarized_counts_for_hw = heat_wave_day_def.groupby(['NIS_Code', 'YEAR'])[['heatwave_boolean']].sum().reset_index()

for arr in arrons_iter:
  for yrs in years_iter:
    ix = hw_count_df.index[hw_count_df['arron'] == arr].tolist()[0]
    val = summarized_counts_for_hw[(summarized_counts_for_hw.NIS_Code == arr) & (summarized_counts_for_hw.YEAR == yrs)]['heatwave_boolean'].tolist()[0]
    try:
      val_i = summarized_temp_for_hw_intensity[(summarized_temp_for_hw_intensity.NIS_Code == arr) & (summarized_temp_for_hw_intensity.YEAR == yrs)]['tempdiff_max_p95'].tolist()[0]
    except:
      val_i = 0
    hw_count_df.loc[ix , yrs] = val
    hw_int_df.loc[ix, yrs] = val_i
    

In [38]:
heat_wave_intensity[heat_wave_intensity.YEAR==1980]

,NIS_Code,Nom_arrondissement,YEAR,TEMPERATURE_MAX,TEMPERATURE_AVG,p95,heatwave_boolean,tempdiff_max_p95
207,41000,Aalst,1980,30.0,23.9,26.125,1,3.875
15183,11000,Antwerpen,1980,30.0,23.9,26.125,1,3.875
30156,81000,Arlon,1980,24.5,17.1,24.425,1,0.075
30157,81000,Arlon,1980,24.7,18.9,24.425,1,0.275
30158,81000,Arlon,1980,26.6,20.5,24.425,1,2.175
45135,51000,Ath,1980,29.4,23.4,25.525,1,3.875
60111,45000,Audenarde,1980,30.0,23.9,26.125,1,3.875
75084,82000,Bastogne,1980,24.5,17.1,24.425,1,0.075
75085,82000,Bastogne,1980,24.7,18.9,24.425,1,0.275
75086,82000,Bastogne,1980,26.6,20.5,24.425,1,2.175


In [ ]:
hw_count_df.head(5)

,arron,1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,41000,1,0,0,0,2,0,4,0,1,...,0,1,3,0,1,1,1,1,3,5
1,11000,1,0,0,0,2,0,4,0,1,...,0,1,3,0,1,1,1,1,3,5
2,81000,3,0,2,0,2,0,4,0,1,...,0,1,0,2,2,2,0,4,2,5
3,51000,1,0,0,0,2,0,4,0,1,...,0,1,0,1,1,1,0,2,2,5
4,45000,1,0,0,0,2,0,4,0,1,...,0,1,3,0,1,1,1,1,3,5


In [ ]:
ix

43

save the file with heat wave intensities

In [ ]:
from google.colab import files

hw_int_df.to_csv('hw_int_df.csv', encoding = 'utf-8-sig', index=False) 
files.download('hw_int_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

save the file with heat wave frequencies

In [ ]:
 
hw_count_df.to_csv('hw_freq_df.csv', encoding = 'utf-8-sig', index=False) 
files.download('hw_freq_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the following part is from the first code of Mario about merging about mortality and locations, I did not use it for the maps

In [ ]:
heat_wave_week = heat_wave_day_def.groupby(['NIS_Code', 'Nom_arrondissement', 'YEAR_WEEK'])['heatwave_boolean'].sum().reset_index()
heat_wave_week['heatwave_week_boolean'] = (heat_wave_week['heatwave_boolean'] > 0).astype(int)
heat_wave_week

,NIS_Code,Nom_arrondissement,YEAR_WEEK,heatwave_boolean,heatwave_week_boolean
0,11000,Antwerpen,1999-052,0,0
1,11000,Antwerpen,2000-001,0,0
2,11000,Antwerpen,2000-002,0,0
3,11000,Antwerpen,2000-003,0,0
4,11000,Antwerpen,2000-004,0,0
...,...,...,...,...,...
45975,93000,Philippeville,2019-049,0,0
45976,93000,Philippeville,2019-050,0,0
45977,93000,Philippeville,2019-051,0,0
45978,93000,Philippeville,2019-052,0,0


In [ ]:
heat_wave_week['heatwave_week_boolean'].sum()

854

In [ ]:
mortality_heat_wave = pd.merge(heat_wave_week[['YEAR_WEEK', 'NIS_Code', 'Nom_arrondissement', 'heatwave_week_boolean']], mortality_data[['ARRON', 'YEAR_WEEK', 'COD', 'N_MASK']], left_on = ['NIS_Code', 'YEAR_WEEK'], right_on = ['ARRON', 'YEAR_WEEK'], how = 'inner')
mortality_heat_wave

,YEAR_WEEK,NIS_Code,Nom_arrondissement,heatwave_week_boolean,ARRON,COD,N_MASK
0,2000-001,11000,Antwerpen,0,11000,external,NaN
1,2000-001,11000,Antwerpen,0,11000,natural,31.0
2,2000-002,11000,Antwerpen,0,11000,external,13.0
3,2000-002,11000,Antwerpen,0,11000,natural,263.0
4,2000-003,11000,Antwerpen,0,11000,external,11.0
...,...,...,...,...,...,...,...
81137,2019-050,93000,Philippeville,0,93000,external,NaN
81138,2019-050,93000,Philippeville,0,93000,natural,13.0
81139,2019-051,93000,Philippeville,0,93000,natural,15.0
81140,2019-052,93000,Philippeville,0,93000,external,NaN


Save dataset

In [ ]:
import os
directory = "out/"
if not os.path.exists(directory):
    os.makedirs(directory)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>